In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.express as px
import plotly

from bread.vis import *
from bread.data import Features, SegmentationFile, Microscopy
from bread.algo.lineage import LineageGuesser, LineageGuesserNN, LineageGuesserML, LineageGuesserNearestCell, accuracy
from bread.data import Lineage

In [28]:
def extract_features(segmentation, guesser):
    candidate_features = pd.DataFrame()
    bud_ids, time_ids = segmentation.find_buds().bud_ids, segmentation.find_buds().time_ids
    f_list = []
    for i, (bud_id, time_id) in enumerate(zip(bud_ids, time_ids)):
        frame_range = guesser.segmentation.request_frame_range(
            time_id, time_id + guesser.num_frames)
        num_frames_available = guesser.num_frames
        if len(frame_range) < 2:
            # not enough frames
            continue
        if len(frame_range) < guesser.num_frames:
            num_frames_available = len(frame_range)

        # check the bud still exists !
        for time_id_ in frame_range:
            if bud_id not in guesser.segmentation.cell_ids(time_id_):
                # bud has disappeared
                continue
        selected_times = [i for i in range(
            time_id, time_id + num_frames_available)]
        try:
            candidate_parents = guesser._candidate_parents(
                time_id, nearest_neighbours_of=bud_id)
            for c_id, candidate in enumerate(candidate_parents):
                features, f_list = guesser._get_features(
                    bud_id, candidate, time_id, selected_times)
                new_row = {'bud_id': bud_id, 'candid_id': candidate, 'time_id': time_id}
                new_row.update(features)
                new_df= pd.DataFrame(new_row, index=[0])
                candidate_features = pd.concat([candidate_features, new_df])
        except Exception as e:
            print("Error for bud {} at time {} with candidate {}: {}".format(
                bud_id, time_id, candidate, e))
    return candidate_features, f_list

In [3]:
args = {'fov': 0, 'bud_distance_max': 8, 'num_frames_refractory': 0, 'num_frames': 8, 'output_file': 'lineage.csv'}
selected_keys =['dist_0','dist_max','dist_min','dist_std', 'dist_max_min','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first', 'position_bud_last_minus_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

In [4]:
# extract features

data_path = '../../../data/'
features_all = pd.DataFrame()
features_all_normalized_minmax = pd.DataFrame()
features_all_normalized_zscore = pd.DataFrame()

features_all = pd.DataFrame()
for colony in range(0,6):
    print(f'Processing colony {colony}')
    segmentation_file = os.path.join(data_path, f'colony00{colony}_segmentation.h5')
    segmentation = SegmentationFile.from_h5(segmentation_file).get_segmentation('FOV'+str(args['fov']))
    guesser = LineageGuesserNN(
        segmentation=segmentation,
        dist_threshold=args['bud_distance_max'],
        num_frames_refractory=args['num_frames_refractory'],
        num_frames=args['num_frames'],
    )
    features, f_list = extract_features(segmentation, guesser)
    features['colony'] = colony
    features_all = pd.concat([features_all, features])
    
# normalize features min-max
X = features_all[selected_keys]
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)
features_all_normalized_minmax = features_all.copy()
features_all_normalized_minmax[selected_keys] = X_norm

# normalize features z-score
X = features_all[selected_keys]
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)
features_all_normalized_zscore = features_all.copy()
features_all_normalized_zscore[selected_keys] = X_norm


Processing colony 0
No model was provided, using the default model


/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning: OpenCV returned multiple contours, 2 found.
  warnings.warn(Contour.MultipleContoursWarning(len(contours_cv)))
/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:219: BreadWarning: cell #44 does not have nearest neighbours with a distance less than 8, and flexible_threshold is False.
  warnings.warn(BreadWarning(f'cell #{nearest_neighbours_of} does not have nearest neighbours with a distance less than {self.dist_threshold}, and flexible_threshold is {self.flexible_nn_threshold}.'))


Error for bud 44 at time 74 with candidate 33: No candidate parents have been found for in frame #74.


/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning: OpenCV returned multiple contours, 4 found.
  warnings.warn(Contour.MultipleContoursWarning(len(contours_cv)))


Processing colony 1
No model was provided, using the default model
Processing colony 2
No model was provided, using the default model
Processing colony 3
No model was provided, using the default model
Error for bud 73 at time 163 with candidate 53: Unable to find cell_id=53 at time_id=167 in the segmentation.


/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:219: BreadWarning: cell #77 does not have nearest neighbours with a distance less than 8, and flexible_threshold is False.
  warnings.warn(BreadWarning(f'cell #{nearest_neighbours_of} does not have nearest neighbours with a distance less than {self.dist_threshold}, and flexible_threshold is {self.flexible_nn_threshold}.'))


Error for bud 77 at time 167 with candidate 13: No candidate parents have been found for in frame #167.
Processing colony 4
No model was provided, using the default model
Error for bud 40 at time 123 with candidate 15: Unable to find cell_id=40 at time_id=124 in the segmentation.
Error for bud 50 at time 136 with candidate 34: Unable to find cell_id=34 at time_id=141 in the segmentation.
Error for bud 55 at time 137 with candidate 34: Unable to find cell_id=34 at time_id=141 in the segmentation.
Error for bud 82 at time 154 with candidate 32: Unable to find cell_id=82 at time_id=161 in the segmentation.
Error for bud 87 at time 157 with candidate 60: Unable to find cell_id=60 at time_id=161 in the segmentation.
Error for bud 105 at time 169 with candidate 67: Unable to find cell_id=67 at time_id=170 in the segmentation.


/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:219: BreadWarning: cell #114 does not have nearest neighbours with a distance less than 8, and flexible_threshold is False.
  warnings.warn(BreadWarning(f'cell #{nearest_neighbours_of} does not have nearest neighbours with a distance less than {self.dist_threshold}, and flexible_threshold is {self.flexible_nn_threshold}.'))


Error for bud 114 at time 173 with candidate 97: No candidate parents have been found for in frame #173.
Processing colony 5
No model was provided, using the default model


In [5]:
# get the lineage ground truth
lin_gt_all = pd.DataFrame()
for colony in range(0,6):
    lineage_gt_path = os.path.join(data_path, f'colony00{colony}_lineage_gt.csv')
    lin_truth = pd.read_csv(lineage_gt_path)
    lin_truth['colony'] = colony
    lin_gt_all = pd.concat([lin_gt_all, lin_truth])
lin_gt_all.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)
lin_gt_all.reset_index(inplace=True, drop=True)


In [6]:
lin_gt_all


,parent_GT,bud_id,time_index,colony
0,-1,1,0,0
1,-1,2,0,0
2,-1,3,0,0
3,-1,4,0,0
4,1,5,18,0
...,...,...,...,...
867,22,34,164,5
868,13,35,166,5
869,18,36,169,5
870,3,37,175,5


In [7]:
# add is_parent column
# Start by merging the two dataframes based on the colony and bud_id columns
def merge(features_all, lin_gt_all):
    merged = pd.merge(features_all, lin_gt_all, on=['colony', 'bud_id'], how='left')

    # Define a function that checks if a candidate is the true parent or not
    def is_true_parent(row):
        return row['parent_GT'] == row['candid_id']

    # Apply the function to each row of the merged dataframe to obtain a new column
    merged['is_true_parent'] = merged.apply(is_true_parent, axis=1)
    return merged
merged_features_all = merge(features_all, lin_gt_all)
merged_features_all_normalized_minmax = merge(features_all_normalized_minmax, lin_gt_all)
merged_features_all_normalized_zscore = merge(features_all_normalized_zscore, lin_gt_all)
merged_features_all = merged_features_all.loc[merged_features_all['parent_GT'] > 0]
merged_features_all_normalized_minmax = merged_features_all_normalized_minmax.loc[merged_features_all_normalized_minmax['parent_GT'] > 0]
merged_features_all_normalized_zscore = merged_features_all_normalized_zscore.loc[merged_features_all_normalized_zscore['parent_GT'] > 0]


In [50]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def save_all_feature_distributions(df, feature_list, title):
    fig = make_subplots(
        rows=len(feature_list),
        cols=1,
        print_grid=True,
        subplot_titles=feature_list
    )

    for i, feature in enumerate(feature_list):
        true_data = df[df['is_true_parent'] == True][feature].to_numpy()
        false_data = df[df['is_true_parent'] == False][feature].to_numpy()
        fig.add_trace(
            go.Histogram(
                x=true_data,
                marker=dict(color='blue'),
                opacity=0.75,
                name='True',
            ),
            col=1,
            row=i + 1
        )
        fig.add_trace(
            go.Histogram(
                x=false_data,
                marker=dict(color='red'),
                opacity=0.75,
                name='False',
            ),
            col=1,
            row=i + 1
        )
    fig.update_layout(
    autosize=True,
    height=20000
    )
    fig.update_layout(barmode='overlay')

    fig.update_layout(title=title)

    fig.update_xaxes(rangeslider=dict(visible=False))
    fig.write_html(title + '.html')

In [54]:
save_all_feature_distributions(merged_features_all, selected_keys, 'features_all_dist_threshold_8_num_frames_8')
save_all_feature_distributions(merged_features_all_normalized_minmax, selected_keys, 'features_all_normalized_minmax_dist_threshold_8_num_frames_8')
save_all_feature_distributions(merged_features_all_normalized_zscore, selected_keys, 'features_all_normalized_zscore_dist_threshold_8_num_frames_8')

This is the format of your plot grid:
[ (1,1) x,y      ]
[ (2,1) x2,y2    ]
[ (3,1) x3,y3    ]
[ (4,1) x4,y4    ]
[ (5,1) x5,y5    ]
[ (6,1) x6,y6    ]
[ (7,1) x7,y7    ]
[ (8,1) x8,y8    ]
[ (9,1) x9,y9    ]
[ (10,1) x10,y10 ]
[ (11,1) x11,y11 ]
[ (12,1) x12,y12 ]
[ (13,1) x13,y13 ]
[ (14,1) x14,y14 ]
[ (15,1) x15,y15 ]
[ (16,1) x16,y16 ]
[ (17,1) x17,y17 ]
[ (18,1) x18,y18 ]
[ (19,1) x19,y19 ]
[ (20,1) x20,y20 ]
[ (21,1) x21,y21 ]

This is the format of your plot grid:
[ (1,1) x,y      ]
[ (2,1) x2,y2    ]
[ (3,1) x3,y3    ]
[ (4,1) x4,y4    ]
[ (5,1) x5,y5    ]
[ (6,1) x6,y6    ]
[ (7,1) x7,y7    ]
[ (8,1) x8,y8    ]
[ (9,1) x9,y9    ]
[ (10,1) x10,y10 ]
[ (11,1) x11,y11 ]
[ (12,1) x12,y12 ]
[ (13,1) x13,y13 ]
[ (14,1) x14,y14 ]
[ (15,1) x15,y15 ]
[ (16,1) x16,y16 ]
[ (17,1) x17,y17 ]
[ (18,1) x18,y18 ]
[ (19,1) x19,y19 ]
[ (20,1) x20,y20 ]
[ (21,1) x21,y21 ]

This is the format of your plot grid:
[ (1,1) x,y      ]
[ (2,1) x2,y2    ]
[ (3,1) x3,y3    ]
[ (4,1) x4,y4    ]
[ (5,1) x5

In [52]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def save_all_feature_violin(df, feature_list, title):
    fig = make_subplots(
        rows=int(len(feature_list)/5)+1,
        cols=5,
        print_grid=True,
        subplot_titles=feature_list
    )

    for i, feature in enumerate(feature_list):
        for state in [True, False]:
            fig.add_trace(go.Violin(x=df['is_true_parent'][df['is_true_parent'] == state],
                            y=df[feature][df['is_true_parent'] == state],
                            name=state,
                            box_visible=True,
                            meanline_visible=True),row=int((i/5)+1), col=i%5+1)
                
    fig.update_layout(
        autosize=True,
        height=3000
        )
    fig.update_layout(barmode='overlay')

    fig.update_layout(title=title)


    fig.update_layout(showlegend=False)
    fig.write_html(title + '.html')

In [53]:
save_all_feature_violin(merged_features_all, selected_keys, 'violon_features_all_dist_threshold_8_num_frames_8')
save_all_feature_violin(merged_features_all_normalized_minmax, selected_keys, 'violon_features_all_normalized_minmax_dist_threshold_8_num_frames_8')
save_all_feature_violin(merged_features_all_normalized_zscore, selected_keys, 'violon_features_all__normalized_zscore_dist_threshold_8_num_frames_8')

This is the format of your plot grid:
[ (1,1) x,y     ]  [ (1,2) x2,y2   ]  [ (1,3) x3,y3   ]  [ (1,4) x4,y4   ]  [ (1,5) x5,y5   ]
[ (2,1) x6,y6   ]  [ (2,2) x7,y7   ]  [ (2,3) x8,y8   ]  [ (2,4) x9,y9   ]  [ (2,5) x10,y10 ]
[ (3,1) x11,y11 ]  [ (3,2) x12,y12 ]  [ (3,3) x13,y13 ]  [ (3,4) x14,y14 ]  [ (3,5) x15,y15 ]
[ (4,1) x16,y16 ]  [ (4,2) x17,y17 ]  [ (4,3) x18,y18 ]  [ (4,4) x19,y19 ]  [ (4,5) x20,y20 ]
[ (5,1) x21,y21 ]  [ (5,2) x22,y22 ]  [ (5,3) x23,y23 ]  [ (5,4) x24,y24 ]  [ (5,5) x25,y25 ]

This is the format of your plot grid:
[ (1,1) x,y     ]  [ (1,2) x2,y2   ]  [ (1,3) x3,y3   ]  [ (1,4) x4,y4   ]  [ (1,5) x5,y5   ]
[ (2,1) x6,y6   ]  [ (2,2) x7,y7   ]  [ (2,3) x8,y8   ]  [ (2,4) x9,y9   ]  [ (2,5) x10,y10 ]
[ (3,1) x11,y11 ]  [ (3,2) x12,y12 ]  [ (3,3) x13,y13 ]  [ (3,4) x14,y14 ]  [ (3,5) x15,y15 ]
[ (4,1) x16,y16 ]  [ (4,2) x17,y17 ]  [ (4,3) x18,y18 ]  [ (4,4) x19,y19 ]  [ (4,5) x20,y20 ]
[ (5,1) x21,y21 ]  [ (5,2) x22,y22 ]  [ (5,3) x23,y23 ]  [ (5,4) x24,y24 ]  [

In [ ]:
import plotly.express as px


def plot_feature_distribution(df, feature_name, title):
    true_data = df[df['is_true_parent'] == True][feature_name].to_numpy()
    false_data = df[df['is_true_parent'] == False][feature_name].to_numpy()
    df =pd.DataFrame(dict(
        series = np.concatenate(([True]*len(true_data), [False]*len(false_data))), 
        data = np.concatenate((true_data,false_data))
    ))
    fig = px.histogram(df, x="data", color="series", barmode="overlay", opacity=0.75, color_discrete_map = {False:'red',True:'blue'})
    fig.update_layout(title=title+'_'+feature_name)
    fig.show()

# Plot the distribution of the features for the true parent vs not real parent
for  feature in selected_keys:
    plot_feature_distribution(merged_features_all, feature, 'features_all')
    plot_feature_distribution(merged_features_all_normalized_minmax, feature, 'features_all_normalized_minmax')
    plot_feature_distribution(merged_features_all_normalized_zscore, feature, 'features_all_normalized_zscore')
# plot_feature_distribution(merged_features_all, 'dist_0', 'features_all')
# plot_features_distribution(merged_features_all_normalized_minmax, selected_keys, 'features_all_normalized_minmax')
# plot_features_distribution(merged_features_all_normalized_zscore, selected_keys, 'features_all_normalized_zscore')



In [27]:
# histogram for each feature
df = merged_features_all
for feature_name in selected_keys:
    title = 'features_name'
    true_data = df[df['is_true_parent'] == True][feature_name].to_numpy()
    false_data = df[df['is_true_parent'] == False][feature_name].to_numpy()
    df_f =pd.DataFrame(dict(
        series = np.concatenate(([True]*len(true_data), [False]*len(false_data))), 
        data = np.concatenate((true_data,false_data))
    ))
    fig = px.histogram(df_f, x="data", color="series", barmode="overlay", opacity=0.75, color_discrete_map = {False:'red',True:'blue'})
    fig.update_layout(title=title+'_'+feature_name)
    fig.show()
fig.write_html('test' + '.html')

In [57]:
# change time_frame and bud_distance_max
for bud_distance_max in [8,16,128]:
    for time_frame in [4,8]:
        # extract features
        args = {'fov': 0, 'bud_distance_max': bud_distance_max, 'num_frames_refractory': 0, 'num_frames': time_frame, }

        features_all_normalized_minmax = pd.DataFrame()
        features_all_normalized_zscore = pd.DataFrame()
        features_all = pd.DataFrame()
        for colony in range(0,6):
            print(f'Processing colony {colony}')
            segmentation_file = os.path.join(data_path, f'colony00{colony}_segmentation.h5')
            segmentation = SegmentationFile.from_h5(segmentation_file).get_segmentation('FOV'+str(args['fov']))
            guesser = LineageGuesserNN(
                segmentation=segmentation,
                dist_threshold=args['bud_distance_max'],
                num_frames_refractory=args['num_frames_refractory'],
                num_frames=args['num_frames'],
            )
            features, f_list = extract_features(segmentation, guesser)
            features['colony'] = colony
            features_all = pd.concat([features_all, features])
            
        # normalize features min-max
        X = features_all[selected_keys]
        scaler = MinMaxScaler()
        X_norm = scaler.fit_transform(X)
        features_all_normalized_minmax = features_all.copy()
        features_all_normalized_minmax[selected_keys] = X_norm

        # normalize features z-score
        X = features_all[selected_keys]
        scaler = StandardScaler()
        X_norm = scaler.fit_transform(X)
        features_all_normalized_zscore = features_all.copy()
        features_all_normalized_zscore[selected_keys] = X_norm
        # merge them with ground truth
        merged_features_all = merge(features_all, lin_gt_all)
        merged_features_all_normalized_minmax = merge(features_all_normalized_minmax, lin_gt_all)
        merged_features_all_normalized_zscore = merge(features_all_normalized_zscore, lin_gt_all)
        merged_features_all = merged_features_all.loc[merged_features_all['parent_GT'] > 0]
        merged_features_all_normalized_minmax = merged_features_all_normalized_minmax.loc[merged_features_all_normalized_minmax['parent_GT'] > 0]
        merged_features_all_normalized_zscore = merged_features_all_normalized_zscore.loc[merged_features_all_normalized_zscore['parent_GT'] > 0]
        # save the plots
        save_all_feature_violin(merged_features_all, selected_keys, 'violin_features_all_dist_threshold_{}_num_frames_{}'.format(bud_distance_max, time_frame))
        save_all_feature_violin(merged_features_all_normalized_minmax, selected_keys, 'violin_features_all_normalized_minmax_dist_threshold_{}_num_frames_{}'.format(bud_distance_max, time_frame))
        save_all_feature_violin(merged_features_all_normalized_zscore, selected_keys, 'violin_features_all__normalized_zscore_dist_threshold_{}_num_frames_{}'.format(bud_distance_max, time_frame))
        save_all_feature_distributions(merged_features_all, selected_keys, 'histogram_features_all_dist_threshold_{}_num_frames_{}'.format(bud_distance_max, time_frame))
        save_all_feature_distributions(merged_features_all_normalized_minmax, selected_keys, 'histogram_features_all_normalized_minmax_dist_threshold_{}_num_frames_{}'.format(bud_distance_max, time_frame))
        save_all_feature_distributions(merged_features_all_normalized_zscore, selected_keys, 'histogram_features_all_normalized_zscore_dist_threshold_{}_num_frames_{}'.format(bud_distance_max, time_frame))


        


Processing colony 0
No model was provided, using the default model


/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning:

OpenCV returned multiple contours, 2 found.

/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:219: BreadWarning:

cell #44 does not have nearest neighbours with a distance less than 8, and flexible_threshold is False.



Error for bud 44 at time 74 with candidate 33: No candidate parents have been found for in frame #74.


/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning:

OpenCV returned multiple contours, 4 found.



Processing colony 1
No model was provided, using the default model
Processing colony 2
No model was provided, using the default model
Processing colony 3
No model was provided, using the default model


...

Processing colony 1
No model was provided, using the default model
Processing colony 2
No model was provided, using the default model
Processing colony 3
No model was provided, using the default model
Processing colony 4
No model was provided, using the default model
Error for bud 39 at time 122 with candidate 24: Unable to find cell_id=24 at time_id=124 in the segmentation.
Error for bud 40 at time 123 with candidate 1: Unable to find cell_id=40 at time_id=124 in the segmentation.
Error for bud 56 at time 138 with candidate 34: Unable to find cell_id=34 at time_id=141 in the segmentation.
Error for bud 90 at time 160 with candidate 48: Unable to find cell_id=48 at time_id=161 in the segmentation.
Error for bud 104 at time 168 with candidate 67: Unable to find cell_id=67 at time_id=170 in the segmentation.
Error for bud 105 at time 169 with candidate 7: Unable to find cell_id=105 at time_id=170 in the segmentation.
Processing colony 5
No model was provided, using the default model
This

/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning:

OpenCV returned multiple contours, 2 found.

/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:219: BreadWarning:

cell #44 does not have nearest neighbours with a distance less than 128, and flexible_threshold is False.



Error for bud 44 at time 74 with candidate 40: No candidate parents have been found for in frame #74.
Error for bud 56 at time 82 with candidate 32: Unable to find cell_id=32 at time_id=85 in the segmentation.
Error for bud 162 at time 118 with candidate 155: Unable to find cell_id=155 at time_id=124 in the segmentation.
Error for bud 175 at time 121 with candidate 155: Unable to find cell_id=155 at time_id=124 in the segmentation.
Error for bud 185 at time 122 with candidate 155: Unable to find cell_id=155 at time_id=124 in the segmentation.
Error for bud 189 at time 123 with candidate 155: Unable to find cell_id=155 at time_id=124 in the segmentation.
Error for bud 242 at time 132 with candidate 179: Unable to find cell_id=179 at time_id=136 in the segmentation.
Error for bud 255 at time 134 with candidate 179: Unable to find cell_id=179 at time_id=136 in the segmentation.


/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning:

OpenCV returned multiple contours, 4 found.



Processing colony 1
No model was provided, using the default model
Processing colony 2
No model was provided, using the default model
Processing colony 3
No model was provided, using the default model
Error for bud 72 at time 163 with candidate 53: Unable to find cell_id=53 at time_id=167 in the segmentation.
Error for bud 73 at time 163 with candidate 53: Unable to find cell_id=53 at time_id=167 in the segmentation.
Processing colony 4
No model was provided, using the default model
Error for bud 34 at time 117 with candidate 24: Unable to find cell_id=24 at time_id=124 in the segmentation.
Error for bud 35 at time 118 with candidate 24: Unable to find cell_id=24 at time_id=124 in the segmentation.
Error for bud 36 at time 119 with candidate 24: Unable to find cell_id=24 at time_id=124 in the segmentation.
Error for bud 39 at time 122 with candidate 24: Unable to find cell_id=24 at time_id=124 in the segmentation.
Error for bud 40 at time 123 with candidate 1: Unable to find cell_id=40